# **MLP** (Multi-Layer Perceptron)
<img src="docs/images/image-5.png" alt="mlp" width="550" />

In [ ]:
import numpy as np

In [1]:
def standardize(X):

    mu = X.mean(axis=0)
    std = X.std(axis=0)
    std = np.where(std == 0, 1, std)

    XS = (X - mu) / std

    return XS

In [ ]:
from sklearn.datasets import load_iris 

iris = load_iris(as_frame=True)
X = iris.data
y = iris.target

# Selecting just 2 classes
X = X.iloc[0:100]
X = X.values

y = y[y != 2]